**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**2. Knapsack Problem**

The Knapsack problem is a popular problem in the fundamentals of Artificial Intelligence and optimization in searching. It involves a list of objects with different prices and weights. The goal of the problem is to fit as much value in objects into the knapsack without going over the capacity.

To take on this problem, optimization is done through searching and trials by the computer.

**3. Dataset**

Dataset is provided by Kaggle.com. It can be downloaded here: https://www.kaggle.com/datasets/warcoder/knapsack-problem?resource=download

The dataset according to Kaggle, is a Public Domain dataset with over 9000 rows in a CSV file.

**Import important libraries**

**1. Group Description**

Group Number: 47

Member Names: Angus Leung

Member Student Numbers: 300110509

In [ ]:
import pandas as pd
import itertools
import numpy as np

import time

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [ ]:
url = "https://raw.githubusercontent.com/angusleung100/CSI4106-A1/main/knapsack_5_items.csv" #provide the link to the raw version of dataset. You *need* to provide a link to *your own* github repository. DO NOT use the link that is provided as an example.

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [ ]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [ ]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [ ]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [ ]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

<ipython-input-8-833fb421dc49>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
<ipython-input-8-833fb421dc49>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
<ipython-input-8-833fb421dc49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

The Generate and Test algorithm seems to be one of the most resource consuming initially but also very accurate as it generates every single possibility and then proceeds to find the best setup.

In [ ]:
def gen_and_test(data):

  # Merge weight and price
  items = []

  for i in range(len(data['Weights'])): # Cycle and combine weights and prices and put into an items list
    items.append([data['Weights'][i], data['Prices'][i]])

  # Variables to return
  best_solution_price = 0 # Current best price
  best_solution = []  # Current best solution

  current_best_weight = 0 # Temp store best weight

  for r in range(len(data['Weights'])): # Create all possible combinations from 1 to r amount of items

    possibilitiesWeights = list(itertools.combinations(items, r+1)) # Create list of possibilities for a combination size r

    #print(possibilitiesWeights)

    for possibility in possibilitiesWeights:  # Test each possibility's weight and price

      # Initial values
      weightSum = 0
      priceSum = 0
      solution = [0,0,0,0,0]

      # load weights selection 1 or 0 into current solution
      for i in range(len(data['Weights'])):

        for pair in possibility:  # Cycle through all pairs in the current possibility calculation

          if pair[0] == data['Weights'][i] and pair[1] == data['Prices'][i]:   # Find position in data input list
            solution[i] = 1 # Change to 1 status

      # Calculate possibility's total weight and price
      for i in range(len(possibility)):
        #print(possibility)
        weightSum += possibility[i][0]
        priceSum += possibility[i][1]

        # set 1 or 0 for weights used from data['Weights']

        if weightSum > data['Capacity']:  # Stop calculating and abort current possibility since overweight
          break

        if priceSum > best_solution_price and weightSum >= current_best_weight and len(possibility) > len(best_solution): # Set current as best solution since it is better than current price-weight
          best_solution_price = priceSum
          current_best_weight = weightSum

          best_solution = solution

  #print(best_solution_price)
  #print(best_solution)
  return best_solution_price, best_solution



In [ ]:
solutions = []
startTime = time.time() # Added start time
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)
endTime = time.time()
genTestRuntime = endTime - startTime
print("Run Time: ", genTestRuntime)

Run Time:  20.30918836593628


In [ ]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 0.07915780949488814


**Your Analysis:**
For my implementation of Gen and Test, I tried to find the option with the most items in the bag and at the best price. I believe it went quite well but was running at over O(n^4) time. I believe that it could have been implemented in a run time lower than O(n^4), but for this experiment, only calculating an accuracy is needed.

Current runtime, which is expected to be long, is ~20.31 seconds.

Based on testing, there are over 5000 rows in the dataset csv and through my implementation, I somehow got a low accuracy score, despite the algorithm generating all possibilities. The accuracy should be closer to 1.

This could be a logic error somewhere in the algorithm implementation.

**5. Greedy Search**

Greedy Search works by calculating the best "deal" (Price per weight) and using that as the benchmark for getting a solution. This works, but of course ignores other potential options and only goes with the ones where the best "deals" are available for item selection.

In [ ]:
def greedy(data):

  profitPerWeight = []

  # Merge weight and price and profit per weight
  items = []

  for i in range(len(data['Weights'])): # Cycle and combine weights and prices and put into an items list
    items.append([data['Weights'][i], data['Prices'][i], data['Prices'][i]/data['Weights'][i]])

  # Sort by smallest to largest profit per weight
  items = sorted(items, key=lambda items: items[2], reverse=True)

  #print(items)

  # Create knapsack of stuff
  best_solution_price = 0
  best_solution = [0, 0, 0, 0, 0]

  capacityLeft = data['Capacity']

  for i in range(len(items)):  # Pick through objects in items
    if items[i][0] <= capacityLeft: # Check if still room in bag
      best_solution[i] = 1  # Toggle solution 0 position to 1
      best_solution_price += items[i][1]  # Add to price
      capacityLeft = capacityLeft - items[i][0] # Subtract item from capacity left

  return best_solution_price, best_solution


------------------------------------------------------------------------------------------------



In [ ]:
solutions_greedy = []
startTime = time.time()
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)
endTime = time.time()
greedyRuntime = endTime - startTime
print("Runtime: ", greedyRuntime)

Runtime:  1.1791775226593018


In [ ]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.8341937443061038


**Your Analysis:**

------------------------------------------------------------------------------------------------

For Greedy Search, it seems to be a bit more accurate for solutions.

Due to the lack of connectivity of lists for weights and prices, they had to be merged and merged again with the ratio of price per weight. Of course, this only took O(n) and the entire algorithm itself only took O(n), making it relatively fast compared to the previous Generate and Test.

The runtime for Greedy is ~1.18 seconds.

It seems to also be quite accurate as I did not manage to mess up the logic implementation for Greedy as I somehow did with Generate and Test and got an accuracy closer to 1.

**6. Simulated Annealing**

Simulated Annealing works like a map/graph system that moves a general "cursor" aka the testing step around the data bounds to find the best "position", which should give the ideal price and weights.

In [ ]:
import random
import math

from numpy import asarray, exp
from numpy.random import randn, rand, seed

# With the help of Machine Learning Mysteries to calculate a score (See references below)

def objective(step):  # Objective score
  return step[0] ** 2

def simulated_annealing(data, N, initial_temperature, cooling_rate):

  # Note: We are moving weights and prices together to match up their indice values for comparing
  items = []

  # Merge weights and prices
  for i in range(len(data['Weights'])): # Cycle and combine weights and prices and put into an items list
    items.append([data['Weights'][i], data['Prices'][i], data['Prices'][i]/data['Weights'][i]])

  # Convert to numpy array to be worked on according to algorithm by MachineLearningMysteries
  weights = asarray([data['Weights']])
  prices = asarray([data['Prices']])

  # Get the starting point
  bestWeights = weights[:, 0] + rand(len(weights)) * (weights[:, 1] - weights[:, 0])  # Create starting point for weights
  bestPrices = prices[:, 0] + rand(len(prices)) * (prices[:, 1] - prices[:, 0])  # Create starting point for prices


  # Simulated Annealing

  #Stepping
  bestEvalWeights = objective(bestWeights)
  bestEvalPrices = objective(bestPrices)

  # Set current position
  currentWeights, currentEvalWeights = bestWeights, bestEvalWeights
  currentPrices, currentEvalPrices = bestPrices, bestEvalPrices

  # Run iterations N
  for i in range(N):

    # Create test step
    stepWeights = currentWeights + randn(len(data['Weights'])) * cooling_rate
    stepEvalWeights = objective(stepWeights)

    stepPrices = currentPrices + randn(len(data['Prices'])) * cooling_rate
    stepEvalPrices = objective(stepPrices)

    # check if test step better than best step and meets constraints
    if stepEvalWeights < bestEvalWeights and stepEvalWeights <= 40.0 and stepEvalPrices > bestEvalPrices:
      # Set new best
      bestWeights, bestEvalWeights = stepWeights, stepEvalWeights
      bestPrices, bestEvalPrices = stepPrices, stepEvalPrices

      #print('Weights: >%d f(%s) = %.5f | Prices: >%d f(%s) = %.5f' % (i, stepWeights, stepEvalWeights, i, stepPrices, stepEvalPrices))

    # Calculate difference
    differenceWeights = stepEvalWeights - currentEvalWeights
    differencePrices = stepEvalPrices - currentEvalPrices

    # Calculate temperature
    temperature = initial_temperature / float(i + 1)

    # Calculate metropolis
    metroWeights = exp(-differenceWeights/temperature)
    metroPrices = exp(-differencePrices/temperature)

    # Check if difference and metro values allow change of current to the test step
    if differencePrices < 0 or rand() < metroPrices:
      currentWeights, currentEvalWeights = stepWeights, stepEvalWeights
      currentPrices, currentEvalPrices = stepPrices, stepEvalPrices

  # Assign best for return
  best_solution_price = bestEvalPrices
  best_solution = bestWeights

  #print(best_solution_price, best_solution)

  return best_solution_price, best_solution


In [ ]:
solutions_sa = []
startTime = time.time()
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)
endTime = time.time()
saRuntime = endTime - startTime
print("Runtime: ", saRuntime)

<ipython-input-130-3cf4aa97fb7f>:64: RuntimeWarning: overflow encountered in exp
  metroWeights = exp(-differenceWeights/temperature)
<ipython-input-130-3cf4aa97fb7f>:65: RuntimeWarning: overflow encountered in exp
  metroPrices = exp(-differencePrices/temperature)


Runtime:  3.9890849590301514


In [ ]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.0004048992813037757


**Your Analysis:**

------------------------------------------------------------------------------------------------
Based on the running of the implemented simulated annealing algorithm, it holds a very very low accuracy of ~0.0004, which I can confidently say is a problem on logic implementation and adapting of the algorithm examples on MachineLearningMysteries.

The simulated annealing runs on a 2 axis plotting system for the step testing, the weights and the prices, and apparently it was implemented incorrrectly, as:
1. The best solutions in debugging were decimal values despite items being whole items
2. The best solution prices were also decimals and lacked any similarity to the best prices in the data set rows.

Runtime, however, was ~3.99 seconds, making it a decent algorithm to run if implemented correctly and runs in O(n).

**7. Genetic Algorithm**

The Genetic Algorithm is based on a swapping and mutation component that casually changes the values of each index until a good solution is found. This requires looping through a parent and its children since derivatives of a parent need to be tested with different mutations to see whether any mutation is better.

In [ ]:
import random
from numpy.random import rand, randint

def calculate_fitness(ind, prices, weights, capacity):

  fitness = 0
  totalWeight = 0
  totalPrice = 0

  for i in range(len(ind)): # Calculate total weight

    # My version of calculating a fitness


    if ind[i] == 1: # Check if 1 position in individual sample
      totalWeight += weights[i]
      totalPrice += prices[i]

  if totalWeight <= capacity: # Check if under or equal capacity
    fitness += 1  # Raise fitness for being under or equal
    fitness += totalPrice # Raise fitness based on price as well

  return fitness

def crossover(parent1, parent2, cross_rate):
  child1, child2 = parent1.copy(), parent2.copy()

  if rand() < cross_rate:
    point = randint(1, len(parent1)-2)

    # Crossover
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]

  return child1, child2

def mutation(child, mut_rate):
  for i in range(len(child)):
    # check for a mutation
    if rand() < mut_rate:
      # flip the bit
      child[i] = 1 - child[i]

  return child

def selection(population, fitnessScores, size):
 # first random selection
 selection_ix = randint(len(population))
 for ix in randint(0, len(population), size-1):
  # check if better (e.g. perform a tournament)
  if fitnessScores[ix] < fitnessScores[selection_ix]:
    selection_ix = ix
 return population[selection_ix]

def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):

  # Generate initial population
  population = []

  for i in range(population_size):

    ind = []

    for d in range(len(data['Weights'])):
      ind.append(randint(0,1))

    population.append(ind)

  # Best price
  best = 0
  bestEval = calculate_fitness(population[0], data['Prices'], data['Weights'], data['Capacity'])

  for i in range(num_generations):  # Run iterations

    fitnessScores = [calculate_fitness(population[i], data['Prices'], data['Weights'], data['Capacity']) for ind in population]

    for n in range(population_size):

      if fitnessScores[n] < bestEval:
        best, bestEval = population[n], fitnessScores[n]

        #print(">%d, new best f(%s) = %f" % (i,  population[i], fitnessScores[i]))

    select = [selection(population, fitnessScores, tournament_size) for _ in range(num_generations)]

    children = []

    for n in range(0, len(population), 2):
      parent1, parent2 = select[n], select[n+1]

      for cross in crossover(parent1, parent2, cross_rate):
        mutation(cross, mut_rate)
        children.append(cross)
    population = children


  best_solution = best
  best_solution_price = bestEval

  #print(best_solution_price)



  return best_solution_price, best_solution

In [ ]:
solutions_ga = []
startTime = time.time()
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.1, cross_rate = 0.7, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)
endTime = time.time()
gaRuntime = endTime - startTime
print("Runtime: ", gaRuntime)

Runtime:  1001.8043310642242


In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.0


**Your Analysis:**

------------------------------------------------------------------------------------------------
Genetic must be one of the slowest algorithms I've implemented despite being in O(n^2) for runtime. I believe this is due to the ambiguous number of iterations that eventually branch out to test more swaps to determine the best setup.

Fitness calculation is relatively straightforward with a custom fitness score function that factors in price and weight. If the weight is over, automatic 0. Price is the most important factor in all the algorithms so it must be part of the fitness score.

Run time currently is ~1001.80 seconds due to the sheer amount of children that I had running with the number of iterations in the population.

I am quite certain that Genetic was somehow implemented wrong but I cannot figure out where it went wrong. I read through algorithms and attempted to implement it to factor in prices in the fitness score but seems that did little affect.

**8. Comparative Study**

description  +  tables/figures

--------------------------------------------------------------------------
After testing all 4 algorithms, these were the results of the experiment:

Table 1: Algorithm results

| Algorithm           | Big-O  | Runtime (secs) | Accuracy |
|---------------------|--------|----------------|----------|
| Generate and Test   | O(n^4) | ~20.31         | ~0.079   |
| Greedy              | O(n)   | ~1.18          | ~0.834   |
| Simulated Annealing | O(n)   | ~3.99          | ~0.0004  |
| Genetic             | O(n^5) | ~1001.80       | ~0.0     |

The most accurate and fastest algorithm was Greedy Search, with a run time of only ~1.18 and an accuracy of 83.4%.

The worst of them all was Genetic with absolutely no accuracy despite factoring in weights and prices for the fitness score. It also had the worst run time of over 16 minutes.

Generate and Test, despite being slow, yielded better run time. I assume Genetic truly reached worst-case runtime and Generate and Test with O(n^4) did not reach worst case.

Simulated Annealing held a 0.04% accuracy, which I also assume was due to bad implementation of the algorithm to factor in weights and prices.



**9. Conclusion**

Comment on the empirical study, its results, and give ideas for future work.

--------------------------------------------------------------------------
In conclusion, I think I messed up on 3 algorithms and will need to go to office hours. All these have a degree of complexity and I myself would consider Simulated Annealing to be the one I look forward to using (assuming I implement it right), as it drops a starting step and moves along the dataset arrays.

I've tried stable diffusion in the past for AI generated images and I believe Simulated Annealing is quite close to the concept that SD uses.

I do not expect anyone to run and verify the run time for Genetic Algorithm as it did take a long time to run through all the mutations of the population.

**10 References**

Make sure you provide references to ALL sources used (articles, code, algorithms).

**Generate and Test**
- https://www.geeksforgeeks.org/generate-and-test-search/

**Greedy**
- https://www.geeksforgeeks.org/fractional-knapsack-problem/

**Simulated Annealing**
- https://www.machinelearningplus.com/machine-learning/simulated-annealing-algorithm-explained-from-scratch-python/#google_vignette
- https://machinelearningmastery.com/simulated-annealing-from-scratch-in-python/

**Genetic**
- https://saturncloud.io/blog/genetic-algorithm-on-a-knapsackalike-optimization-problem/#:~:text=In%20the%20case%20of%20a,weight%20capacity%20of%20any%20knapsack.
- https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/

**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".